In [311]:
import pandas as pd
import numpy as np
import requests
import os
from lxml import html
from lxml import etree
from IPython.display import display

In [247]:
def identify_letter_urls(blog_url):
    response = requests.get(blog_url)
    tree = html.fromstring(response.content)
    hrefs = tree.xpath('//dt[@class="entry-title"]/a[@class="subj-link"]')
    hrefs = filter(lambda href: href.text.startswith("Письмо."), hrefs)
    pages_to_visit = map(lambda href: href.get('href'), hrefs)
    return list(pages_to_visit)

In [248]:
pages_to_visit = identify_letter_urls("https://evo-lutio.livejournal.com/?tag=evolutiolab")

In [281]:
def extract_letter(post_url):
    response = requests.get(post_url)
    tree = html.fromstring(response.content)
    article = tree.xpath('//div[@class="b-singlepost-bodywrapper"]/article')[0]
    lj_authors = article.xpath("span/a/b/text()")
    lj_author = str(lj_authors[0])
    lj_author2 = str(lj_authors[1]) if len(lj_authors) > 1 else None

    lj_author_cited = False
    text = u''
    for el in article.itertext():
        element_text = str(el)
        if (element_text == lj_author2):
            return text
        if (element_text == lj_author):
            lj_author_cited = True
            continue
        if (not lj_author_cited):
            continue
        if (element_text.startswith("(")):
            continue
        text += element_text + '\n'

    return text

In [270]:
from sets import Set
delta = 40
letter_urls = Set()
for page in range(0, 20):
    prev_size = len(letter_urls)
    letter_urls.update(Set(identify_letter_urls("https://evo-lutio.livejournal.com/?tag=evolutiolab&skip=" + str(delta*page))))
    if (prev_size == len(letter_urls)):
        print("Stuck at page " + str(page) + " quitting")
        break;
    else:
        print("Got " + str(len(letter_urls)) + " letter urls now")

Got 46 letter urls now
Got 81 letter urls now
Got 120 letter urls now
Got 160 letter urls now
Got 195 letter urls now
Got 229 letter urls now
Got 264 letter urls now
Got 300 letter urls now
Got 333 letter urls now
Got 356 letter urls now
Stuck at page 10 quitting


In [304]:
mkdir -p letters

In [276]:
letter_urls = list(letter_urls)
f = open("letter_urls.txt", "w")
f.write("\n".join(map(lambda x: str(x), letter_urls)))

In [335]:
import re
for letter_url in letter_urls:
    letter_id = re.findall("([0-9]+).html", letter_url)[0]
    letter_path = "letters/{letter_id}.txt".format(letter_id=letter_id)
    if (os.path.exists(letter_path)):
        print("Letter #{letter_id} already extracted - skipping".format(letter_id=letter_id))
        continue

    letter_text = extract_letter(letter_url)
    with open(letter_path, "w") as f:        
        f.write(letter_text)
    
    print("Extracted letter {letter_id}: {length} length".format(letter_id=letter_id, length=len(letter_text)))

Letter #567581 already extracted - skipping


In [336]:
!du -hs letters

6.0M	letters


In [337]:
ls letters | head -n1

318972.txt


In [346]:
all_text = '';
for fname in os.listdir("letters"):
    with open('letters/'+fname, 'r') as f:
        all_text += f.read() + '\n'

In [440]:
chars = sorted(list(set(all_text)))
chars.insert(0, '\0')
vocab_size = len(chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [441]:
idx = [char_indices[c] for c in all_text]

In [442]:
len(idx)

5501573

In [443]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

('nb sequences:', 5501534)


In [444]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [445]:
(sentences.shape, next_chars.shape)

((5501532, 40), (5501532, 40))

In [446]:
sentences_sample = sentences[0:50000]
next_chars_sample = next_chars[0:50000]

In [447]:
n_fac = 24

In [448]:
from keras.models import Sequential
from keras.layers import Embedding, Dropout, LSTM, TimeDistributed, Dense, Activation
from keras.optimizers import Adam

model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])

In [449]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [390]:
%time model.fit(sentences_sample, np.expand_dims(next_chars_sample,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
50000/50000 [==============================] - 209s - loss: 1.7503   
CPU times: user 2min 49s, sys: 1min 8s, total: 3min 58s
Wall time: 4min 3s


In [450]:
# consume_less=gpu reports to have spent less time = 119s vs 209s :/
%time model.fit(sentences_sample, np.expand_dims(next_chars_sample,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
50000/50000 [==============================] - 121s - loss: 1.7616   
CPU times: user 1min 54s, sys: 30.8 s, total: 2min 24s
Wall time: 2min 27s


In [468]:
from numpy.random import choice
def print_example(symbols=320):
    seed_string="Здравствуйте, Эволюция."
    for i in range(symbols):
        try:
            x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
            preds = model.predict(x, verbose=0)[0][-1]
            preds = preds/np.sum(preds)
            next_char = choice(chars, p=preds)
            seed_string = seed_string + next_char
        except:            
            print("X " + str(x.shape))
            print("Preds " + str(len(preds)))
            print("Chars " + str(len(chars)))
            print("i " + str(i))
            raise
    print(seed_string)

In [453]:
model.fit(sentences_sample, np.expand_dims(next_chars_sample,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
50000/50000 [==============================] - 121s - loss: 1.2714   


In [454]:
print_example()

Здравствуйте, Эволюция.
На стал я у меня :Чыхор ен сельа бод не хаз кела ток гомуше и не нав денерения опять лоторого вв сраси ра на хотиения в иехил «дс. Вень.
У пусно полилалым. Он ходло рямы, жажесклу�


In [455]:
model.fit(sentences_sample, np.expand_dims(next_chars_sample,-1), batch_size=64, nb_epoch=1)
print_example()

Epoch 1/1
50000/50000 [==============================] - 121s - loss: 1.0868   
Здравствуйте, Эволюция.
Потьпить с антересный невелю. Нет, так? На тро «не опять прямшопе рыла мы симьмились, и остраницуся. . 

 Чсастая что ответила сне, лак-м не раз иткатил, чтосут кад это не довегще �


In [456]:
model.fit(sentences_sample, np.expand_dims(next_chars_sample,-1), batch_size=64, nb_epoch=1)
print_example()

Epoch 1/1
50000/50000 [==============================] - 121s - loss: 0.9418   
Здравствуйте, Эволюция.
Про себя (мне 23, едс находили увобо умадал, я его дат-иние, написал что мружым снова в понр-дения по ваю другую с денте объеки в лаписала воздух Притивинось, тебя пришло!
Простил


In [457]:
model.fit(sentences_sample, np.expand_dims(next_chars_sample,-1), batch_size=64, nb_epoch=1)
print_example()

Epoch 1/1
50000/50000 [==============================] - 121s - loss: 0.8359   
Здравствуйте, Эволюция.

Ссе это были понять встречи, антерес с упать с ним бы вышла не умержёю помучтились встретиться ,же мосяца вообще она извинее не ответила. Через неша обо родиловала пок-ли все �


In [458]:
model.fit(sentences_sample, np.expand_dims(next_chars_sample,-1), batch_size=64, nb_epoch=1)
print_example()

Epoch 1/1
50000/50000 [==============================] - 121s - loss: 0.7554   
Здравствуйте, Эволюция.
Сечером заерал привдах потом стал ла приедет своджели домой и все –такие нет, говорили комплиментов. На привело переписывался на разу. 
Я нипрадямать?
-Волчал по жимнье дела» 


In [461]:
model.fit(sentences_sample, np.expand_dims(next_chars_sample,-1), batch_size=64, nb_epoch=1, validation_split=0.1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 111s - loss: 0.6865 - val_loss: 0.7039


In [462]:
print_example()

Здравствуйте, Эволюция.

Извиняться вечером 9го, познакомились и ушла. Да и все ответила. Разводиться, когда я не реагились е возде другую ком того в зафе. На помторится, когда от мужем потом заехал на �


In [463]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, validation_split=0.05)

Train on 5226455 samples, validate on 275077 samples
Epoch 1/1
5226455/5226455 [==============================] - 12748s - loss: 0.9015 - val_loss: 0.8422


In [464]:
print_example()

Здравствуйте, Эволюция.
Когда я видимо. Ланс его причину. А просто мы тоже и написывались. Спросила М. Не отказался, по-других образованиях, был еду на вопрос, когда все это время пока во всем очень айс�


In [465]:
model.save_weights('lesson4_evo_rnn.h5')

In [466]:
print_example()

Здравствуйте, Эволюция.
Вы потеком деньги свою реакцию. Её месте слушай, квы закрыло длительных любви. Когда мне понял, что он был все же мафии, этот усопенность, блружен не приносит, сколько дней я сд�


In [470]:
print_example(2000)

Здравствуйте, Эволюция.
Мы ощутим о жизни. 
На развернулся собой", довольно очень много человека после вечером ощущение искала «чем переспать и отказываются, в отношениях свои вещами. 
Спасибо Вам в этот раз никаких отношениях рядко благодарну". Мне маме ко мне как вообще ты замечаю: мне стала письмо и раз был лишний долезненной и занялась, мне нужна голоса мне расслабиться сам. Я стараться пошла, тем потащил мой ухаживать.
При этом, забыв его приняться с Отношениях, чтоб он поступил с работы комментатора. Но да, что я замолчала мне лица. Я предложила становится. C.Ты может ему очень быть, что не может выбрать меня, я ему нужно была совсем впервые 9. Но наконец нам сказал – интересованный дом на перенимал, начинаю! Когда он сразу жил за честно целовали эту парну, я сказала. Мол у него секса на других. 
И целует, же, нужно пришла меня посидеть в курсе, но уже театр, от его общения, это очень нормально и по-тому делам, как это сама в такую грудый должни, мне было не притягулось. В общем,